In [94]:
from __future__ import print_function
from apiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
from datetime import datetime, timedelta
import pytz
import yaml
import pprint

In [32]:
stream = open('../calendars.yaml', 'r')
dic = next(yaml.load_all(stream))
pcalendars = {p: {} for p in dic['planning']}
print(pcalendars)

{'1. urgent': {}, '2. important': {}, '3. not important': {}, '4. papers': {}}


In [33]:
# Setup the Calendar API
SCOPES = 'https://www.googleapis.com/auth/calendar.readonly'
store = file.Storage('../credentials.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('calendar', 'v3', http=creds.authorize(Http()))


In [42]:
# Get all Relevnt Calendars
desired_attributes = ['id']
page_token = None
while True:
  calendar_list = service.calendarList().list(pageToken=page_token).execute()
  for calendar_list_entry in calendar_list['items']:
    if calendar_list_entry['summary'] in dic['planning']:
        for att in desired_attributes:
            pcalendars[calendar_list_entry['summary']][att] = calendar_list_entry[att]
  page_token = calendar_list.get('nextPageToken')
  if not page_token:
    break
    
pprint.pprint(pcalendars)


{'1. urgent': {'id': 'is6me76qbdsjri3v1op0ago7ds@group.calendar.google.com'},
 '2. important': {'id': '8ns7fa75fqtb6sn4acn9nvajhc@group.calendar.google.com'},
 '3. not important': {'id': '9dsoaf8r7lmms2r0smaajn2dv8@group.calendar.google.com'},
 '4. papers': {'id': '1u18oiuto6lb2jkpo4o7eg66fg@group.calendar.google.com'}}


In [108]:
# Print next "primary" calendar events
now = datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')

start = datetime(2018, 5, 17, 0, 0, 0, tzinfo=eastern).isoformat()
end = datetime(2018, 5, 17, 23, 30, 0, tzinfo=eastern).isoformat()

print(now)
print(start)
for cal in pcalendars:
    print()
    print(cal)
    id=pcalendars[cal]['id']
    events_result = service.events().list(
        calendarId=id, 
        timeMin=start, 
        timeMax=end,
        maxResults=3, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        ev_start = event['start'].get('dateTime', event['start'].get('date'))
        print(ev_start, event['summary'])


Getting the upcoming 10 events
2018-05-17T14:43:00.235728Z
2018-05-17T00:00:00-04:56

1. urgent
2018-05-16T23:30:00-04:00 other: plan time
2018-05-17T04:00:00-04:00 block
2018-05-17T07:00:00-04:00 block

2. important
2018-05-17T18:00:00-07:00 read

3. not important
No upcoming events found.

4. papers
No upcoming events found.


In [56]:
print(now)
print(datetime.datetime.now(pytz.utc))
print(datetime.datetime.now())

2018-05-17T14:09:34.457471Z
2018-05-17 14:17:15.784356+00:00
2018-05-17 23:17:15.784459


In [90]:
from datetime import datetime, timedelta
from pytz import timezone
import pytz

start_date = "month, date, year"
end_date = "month, date, year"

start_time = "xx:xx"
start_time = "xx:xx"

eastern = timezone('US/Eastern')
start = datetime(2018, 5, 17, 6, 0, 0, tzinfo=eastern)
end = datetime(2018, 5, 17, 20, 0, 0, tzinfo=eastern)

print(start)
print(end)

print(datetime.now(eastern))
# start_year = now.year
# start_day = now.day
# start_month = now.month

# end_year = 2018
# end_day = 20
# end_month = 5

# end = now + timedelta(hours=24*())

# # print("start:", start_month, start_day, start_year)
# print(start)
# print(end)

2018-05-17 06:00:00-04:56
2018-05-17 20:00:00-04:56
2018-05-17 10:35:29.381267-04:00
